# NBA Single-Player Search
**Programmer:** Rahim Hashim<br>
**Date:** Sep 2021<br>
**Description:** Scrape basketball-reference.com by single players.<br>

Data captured includes —<br>
* Player Bios<br>
  * *Body Measurements* (height, weight, age...)
  * *Draft* (round, pick, year...)
  * *Hometown* 
  * *College Team*
* Season<br>
  * *Per Game* (pts/game, reb/game, assists/game...)
  * *Totals* (pts, reb, assists...)
  * *Per 36 Minutes* (pts, reb, assists...)
  * *Per 100 Possessions* (pts, reb, assists...)
  * *Advanced* (PER, OBPM, DBPM, VORP...)
  * *Play-by-Play* (PG%, SG%, And1, Blkd...)
  * *Shooting* (FG% by distance, Dunks, Corner 3s...)
* Game Logs <br>
  * Basic and Advanced stats 
  * Win-Loss
* Play-By-Play
  * Real time elapsed between shots, lineups

***

In [ ]:
%reload_ext autoreload
import os
import sys
import pandas as pd
from pprint import pprint
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict, defaultdict
pd.options.mode.chained_assignment = None  # default='warn'

ROOT = '/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/' #@param ['/content/drive/MyDrive/Projects/nba-prediction-algorithm/NBA-Prediction-Algorithms/']  

def add_helpers():
  '''
  add_helper mounts google drive and adds
  helper functions to the sys.path
  '''

  # if running on juypter/google colab, mount to google drive
  if 'google.colab' in str(get_ipython()): 
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    os.chdir(ROOT)

  helper_dir_path = ROOT + 'helper/'
  sys.path.append(helper_dir_path) # set to path of notebook

add_helpers()

In [14]:
from helper.player_scraper import single_player_scraper

kb_meta, kb_data, kb_gamelogs = single_player_scraper('Kobe Bryant')
kb_meta

Searching for name in: https://www.basketball-reference.com/players/b


StopIteration: 

In [ ]:
#@title Table Select { run: "auto" }

#@markdown Per Game | Totals | Advanced | Per Minute | Per Possession | Adjusted Shooting | Play-By-Play | Shooting | All-Star | Salaries
table_type = "advanced" #@param ["all", "per_game", "totals", "advanced", "per_minute", "per_poss", "adjooting", "pbp", "shooting", "all_star", "all_salaries"] {allow-input: true}

#@markdown Season or Playoff Stats
season_playoffs = "both" #@param ["both", "season", "playoffs"]

#@markdown Include or Exclude Career Data
career_data = "exclude" #@param ["include", "exclude"]

def table_select(df, table_type, season_playoffs):
  df_filtered = df.copy(deep=True)
  if table_type != 'all':
    df_filtered = df[df['data_type'] == table_type]
    df_filtered = df_filtered.dropna(how='all', axis='columns')
  if season_playoffs != 'both':
    df_filtered = df_filtered[df_filtered['season_playoffs'] == season_playoffs]
  if career_data == 'exclude':
    df_filtered = df_filtered[df_filtered['season'] != 'Career']

  return df_filtered

df_table = table_select(kb_data, table_type, season_playoffs)
df_table

In [ ]:
df_table.sort_values(by='ws', ascending=False).head(10)

In [ ]:
kb_gamelogs

In [ ]:
type(kb_gamelogs['gmsc'].iloc[5])

[Play-By-Play Stuff](https://jman4190.medium.com/how-to-accessing-live-nba-play-by-play-data-f24e02b0a976)

In [ ]:
kb_gamelogs